In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
# Define dataset path
data_dir = r"C:\Project_ML\disease_detection\potato-data"

# Load dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    shuffle=True,
    image_size=(256, 256),
    batch_size=32
)

# Get class names
class_names = dataset.class_names
print("Available Categories:", class_names)  # ✅ Check if class names are correct


Found 2263 files belonging to 4 classes.
Available Categories: ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'test_images']


In [4]:
# Function to split dataset
def get_dataset_partitions(ds, train_split=0.8, val_split=0.1, test_split=0.1):
    ds_size = len(ds)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions(dataset)

# Optimize dataset performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)


In [5]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(256, 256),
    layers.Rescaling(1.0/255)
])

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
])


In [6]:
# Define model
input_shape = (256, 256, 3)
num_classes = len(class_names)

model = models.Sequential([
    resize_and_rescale,
    data_augmentation,

    layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Build model
model.build(input_shape=(None, 256, 256, 3))
model.summary()  # ✅ Check model summary


c:\Project_ML\disease_detection\pd\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 896,388 (3.42 MB)

 Trainable params: 896,388 (3.42 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
EPOCHS = 15

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


Epoch 1/15
56/56 ━━━━━━━━━━━━━━━━━━━━ 59s 958ms/step - accuracy: 0.4929 - loss: 1.1194 - val_accuracy: 0.7946 - val_loss: 0.6736
Epoch 2/15
56/56 ━━━━━━━━━━━━━━━━━━━━ 48s 867ms/step - accuracy: 0.7585 - loss: 0.6858 - val_accuracy: 0.8080 - val_loss: 0.6311
Epoch 3/15
56/56 ━━━━━━━━━━━━━━━━━━━━ 47s 836ms/step - accuracy: 0.8216 - loss: 0.5896 - val_accuracy: 0.8929 - val_loss: 0.4376
Epoch 4/15
56/56 ━━━━━━━━━━━━━━━━━━━━ 45s 801ms/step - accuracy: 0.8552 - loss: 0.5008 - val_accuracy: 0.8884 - val_loss: 0.3795
Epoch 5/15
56/56 ━━━━━━━━━━━━━━━━━━━━ 41s 740ms/step - accuracy: 0.8856 - loss: 0.4079 - val_accuracy: 0.8750 - val_loss: 0.4276
Epoch 6/15
56/56 ━━━━━━━━━━━━━━━━━━━━ 45s 805ms/step - accuracy: 0.9044 - loss: 0.3565 - val_accuracy: 0.8661 - val_loss: 0.4755
Epoch 7/15
56/56 ━━━━━━━━━━━━━━━━━━━━ 47s 844ms/step - accuracy: 0.8894 - loss: 0.4021 - val_accuracy: 0.8884 - val_loss: 0.3632
Epoch 8/15
56/56 ━━━━━━━━━━━━━━━━━━━━ 46s 819ms/step - accuracy: 0.8908 - loss: 0.3647 - val_accu

In [31]:
# Evaluate model on test data
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc*100:.2f}%")


8/8 [==============================] - 4s 217ms/step - loss: 0.2037 - accuracy: 0.9095
Test Accuracy: 90.95%


In [32]:
model.save("potato_model.h5")

In [36]:
def predict_image(model, img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * np.max(predictions[0]), 2)

    return predicted_class, confidence

# Test prediction
test_image = r"E:\mlproject\potatos-data\potato-disease\test_images\2fe81ef1-d7ec-4395-b071-c74d5beb4037___RS_Early.B 7242.JPG"  # Replace with actual test image
predicted_label, confidence = predict_image(model, test_image)
print(f"Predicted: {predicted_label} with {confidence}% confidence")


1/1 [==============================] - 0s 38ms/step
Predicted: Potato___Late_blight with 100.0% confidence
